En este experimento se intenta correr el foco de la conversación hacia un pariente cercano del usuario para corroborar a quien le corresponde la información brindada y hacer una correcta recuperación.

In [1]:
import requests
import json
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [5]:
index_name = "conversations"
index_path = "../indexes/conversations"
embeddings = OpenAIEmbeddings()

db = FAISS.load_local(folder_path=index_path, index_name=index_name, embeddings=embeddings)


In [3]:
def gpt35_system_user(system_message:str, user_message:str):
    # Hyper 2
    # POST https://yoizenia.openai.azure.com/openai/deployments/GPT35Turbo/chat/completions

    try:
        response = requests.post(
            url="https://yoizenia.openai.azure.com/openai/deployments/GPT35Turbo/chat/completions",
            params={
                "api-version": "2023-05-15",
                "temperature": "0",
            },
            headers={
                "Content-Type": "application/json",
                "api-key": "bd6603c266fc49389659225a451ff03d",
            },
            data=json.dumps({
                "messages": [
                    {
                        "content": system_message,
                        "role": "system"
                    },
                    {
                        "content":user_message,
                        "role": "user"
                    }
                ]
            })
        )
        choices = response.json()['choices']
        answer = {
            "status_code": response.status_code,
            "text": choices[0]['message'],
        }
        return answer

    except requests.exceptions.RequestException:
        print('HTTP Request failed')
        return None

Se simula una conversación en la que se brinda un dato general y un dato sobre un pariente.

In [9]:
sessions =  [
    {
        "message": {
            "role": "user",
            "content": "Quiero un turno para mi madre"
        },
        "metadata": {
          "session_id": "1",
          "person_id": "20",
          "message_id": "1-1",
          "created_at": "20230825T12:00:00.000Z",
        },
    },
    {
        "message": {
            "role": "system",
            "content": "cual es el nombre de tu madre?"
        },
        "metadata": {
            "session_id": "1",
            "person_id": "20",
            "message_id": "1-2",
            "created_at": "20230825T12:01:00.000Z"
        }
    },
    {
        "message": {
            "role": "user",
            "content": "Alba"
        },
        "metadata": {
          "session_id": "1",
          "person_id": "20",
          "message_id": "1-3",
          "created_at": "20230825T13:00:00.000Z",
        },
    },
    {
        "message": {
            "role": "system",
            "content": "y que edad tiene?"
        },
        "metadata": {
            "session_id": "1",
            "person_id": "20",
            "message_id": "1-4",
            "created_at": "20230825T13:01:00.000Z"
        }
    },
    {
        "message": {
            "role": "user",
            "content": "66"
        },
        "metadata": {
          "session_id": "1",
          "person_id": "20",
          "message_id": "1-5",
          "created_at": "20230825T14:00:00.000Z",
        },
    },
    {
        "message": {
            "role": "system",
            "content": "perfecto, tiene turno para el 30 de agosto a las 15:00"
        },
        "metadata": {
            "session_id": "1",
            "person_id": "20",
            "message_id": "1-6",
            "created_at": "20230825T14:01:00.000Z"
        }
    },
    {
        "message": {
            "role": "user",
            "content": "Cual es la capital de Colombia?"
        },
        "metadata": {
          "session_id": "4",
          "person_id": "20",
          "message_id": "4-1",
          "created_at": "20230825T15:00:00.000Z",
        },
    },
    {
        "message": {
            "role": "system",
            "content": "Bogota"
        },
        "metadata": {
            "session_id": "4",
            "person_id": "20",
            "message_id": "4-2",
            "created_at": "20230825T15:01:00.000Z"
        }
    }

  ]



In [10]:
for session in sessions:
    print(str(session['message']))
    print(session['metadata'])
    db.add_texts(
        texts=[str(session['message'])], 
        metadatas=[session['metadata']],
    )

{'role': 'user', 'content': 'Quiero un turno para mi madre'}
{'session_id': '1', 'person_id': '20', 'message_id': '1-1', 'created_at': '20230825T12:00:00.000Z'}
{'role': 'system', 'content': 'cual es el nombre de tu madre?'}
{'session_id': '1', 'person_id': '20', 'message_id': '1-2', 'created_at': '20230825T12:01:00.000Z'}
{'role': 'user', 'content': 'Alba'}
{'session_id': '1', 'person_id': '20', 'message_id': '1-3', 'created_at': '20230825T13:00:00.000Z'}
{'role': 'system', 'content': 'y que edad tiene?'}
{'session_id': '1', 'person_id': '20', 'message_id': '1-4', 'created_at': '20230825T13:01:00.000Z'}
{'role': 'user', 'content': '66'}
{'session_id': '1', 'person_id': '20', 'message_id': '1-5', 'created_at': '20230825T14:00:00.000Z'}
{'role': 'system', 'content': 'perfecto, tiene turno para el 30 de agosto a las 15:00'}
{'session_id': '1', 'person_id': '20', 'message_id': '1-6', 'created_at': '20230825T14:01:00.000Z'}
{'role': 'user', 'content': 'Cual es la capital de Colombia?'}
{'s

In [13]:
user_input_1 = "que sabes de mi madre?"
filter = {"person_id": "20"}
db_results_1 = db.similarity_search_with_score(query=user_input_1, embeddings=embeddings, filter=filter, k=10)
context_1 = "{ message: " + db_results_1[0][0].page_content + ", metadata: " + str(db_results_1[0][0].metadata) + " }"
print(f"CONTEXT:{context_1}")
system_prompt_1 = f"""
Answer the user's question this message as context.
Previous message: {context_1}
"""
answer_1 = gpt35_system_user(system_message=system_prompt_1, user_message=user_input_1)
answer_1["text"]["content"]

CONTEXT:{ message: {'role': 'system', 'content': 'cual es el nombre de tu madre?'}, metadata: {'session_id': '1', 'person_id': '20', 'message_id': '1-2', 'created_at': '20230825T12:01:00.000Z'} }


'Lo siento, pero no sé nada de tu madre. La pregunta anterior estaba siendo dirigida a un asistente virtual o chatbot. Si hay algo más en lo que pueda ayudarte, por favor házmelo saber.'

El modelo falla en tomar la información brindada en el contexto para elaborar la respuesta.